In [1]:
import pandas as pd

In [5]:
project_id = 'luup-prod' 
city_list = ['osaka','kyoto','shibuya','yokohama']

In [8]:
#ライドデータの取得
def get_df_ride():
  ride_query = '''
    #時間帯ごとの移動の流れを可視化したい
    #変数の型を宣言
    declare from_day date;
    declare to_day date;

    set from_day =date_add(date(current_datetime()),interval -30 day);
    set to_day = date(current_datetime());
    #1日ごとの日付を取得
    WITH days AS (
      SELECT  CAST(DATE_TRUNC(start_ride_datetime,DAY) AS DATE) AS DAY
      FROM  `luup-prod.luup_datawarehouse.rides_completed` 
      WHERE 1=1 
      AND start_ride_datetime >= from_day AND start_ride_datetime < to_day 
      AND end_ride_datetime >= from_day AND end_ride_datetime < to_day 
      AND payment_completed = TRUE
      GROUP BY  DAY
      ORDER BY  DAY
    ),rides AS(
      SELECT  DATE_TRUNC(start_ride_datetime,HOUR) AS TIME,start_port_id,end_port_id,user_id,gross_sales_amount_tax_excluded,payment_amount_tax_excluded,ride_duration_seconds
      FROM  `luup-prod.luup_datawarehouse.rides_completed` 
      WHERE 1=1 
      AND start_ride_datetime >= from_day AND start_ride_datetime < to_day 
      AND end_ride_datetime >= from_day AND end_ride_datetime < to_day 
      AND city_id = '{}' 
      ORDER BY TIME
    ),rides2 AS(
      SELECT*,CAST(SUBSTRING(CAST(TIME AS STRING),1,10) AS DATE) AS DAY
      FROM  rides
    ),rides3 AS(
      SELECT
        TIME,
        start_port_id,port_name AS start_port_name,lat AS start_lat,lng AS start_lng,luup_datawarehouse.convert_address_to_ward(address) AS start_ward,station AS start_station,distance AS start_distance,maximum_device_count AS start_capacity,
        end_port_id,user_id,gross_sales_amount_tax_excluded,payment_amount_tax_excluded,ride_duration_seconds
      FROM rides2 AS A LEFT JOIN `luup-prod.luup_datawarehouse.ports_latest` AS B ON A.start_port_id = B.port_id
    ),rides4 AS(
      SELECT
        TIME,
        start_port_id,end_port_id,
        user_id,gross_sales_amount_tax_excluded,payment_amount_tax_excluded,ride_duration_seconds
      FROM rides3 AS A LEFT JOIN `luup-prod.luup_datawarehouse.ports_latest` AS B ON A.end_port_id = B.port_id
    )
    select * from rides4
    '''.format(city_name)
  df_ride_all = pd.read_gbq(ride_query,project_id)
  return df_ride_all

In [9]:
#30日間の移動データを保存する
for city_name in city_list:
    df_ride = get_df_ride()
    df_ride.to_csv(f'/Users/pakuteiru/Desktop/定期実行/ride/ride_{city_name}.csv',index=False)